In [1]:
from webscraper import Scraper

In [2]:
scraper = Scraper() #call scraper class
scraper.accept_cookies()
scraper.navigate()
scraper.search_bar("final fantasy") #add search keyword here



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been s

Webpage loaded successfully
'Accept Cookies' button clicked
Search bar opened
Search keywords entered
Submit search button clicked - redirected to results


In [3]:
import selenium
from selenium.webdriver import Chrome
from webdriver_manager.chrome import ChromeDriverManager #installs Chrome webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

container = scraper.find_container()
#find many elements that correspond with the XPath - they have to be direct children of the container
#i.e. one level below the container
list_products = container.find_elements(By.XPATH, './/a')

In [4]:
link_list = []
for product in list_products: #iterate through each product
    #print(product.text) #print each product in text format
    link_list.append(product.get_attribute("href"))

In [9]:
from selenium.common.exceptions import NoSuchElementException
import uuid #universally unique id
import time

#add link and product info to dictionary
product_dict = {"Link": [], "Product Name" :[], "Price" : [], "Description" : [], "ID": [], "UUID":[]}
image_dict = {"Link": [], "Product UUID": [], "Image UUID": []}

def retrieve_data():
   #link_list #lists all urls in specified webpage section
   for link in link_list[3:8]: #iterates through links 4 to 9 - FFVIII downloads not included (DOB required)
      scraper.driver.get(link)
      time.sleep(2)
      product_dict["Link"].append(link)
      try: #get product name
         product_name = scraper.driver.find_element(By.XPATH, 
            '//*[@id="responsive-wrapper-title"]/header/h1')
            #shortened form of //*[@id="responsive-wrapper-title"]/header/h1
            #.//h1[@class="product-title"] - works some of the time
         product_dict["Product Name"].append(product_name.text)
         print("Product name obtained")
      except NoSuchElementException: #not all links accessed will have the same attributes
         product_dict["Product Name"].append("N/A")
      
      time.sleep(3)
      try: #get price
         price = scraper.driver.find_element(By.XPATH, './/span[@class = "prices"]')
            #shortened form of //*[@id="main-content"]/article/div[1]/div/div[2]/div[2]/div[1]/div[1]/div/span
         product_dict["Price"].append(price.text)
         print("Price obtained")
      except NoSuchElementException:
         product_dict["Price"].append("N/A")
      
      time.sleep(3)
      try: #get product description
         desc = scraper.driver.find_element(By.XPATH, './/div[@class="tab-pane-content"]')
            #shortened form of //*[@id="desc-collapse"]/div/div
         product_dict["Description"].append(desc.text)
         print("Product description obtained")
      except NoSuchElementException:
         product_dict["Description"].append("N/A")
      
      time.sleep(3)
      try: #get product SKU/ID from URL
         #to get SKU - product details tab needs to be clicked and then an if statement to get the SKU otherwise other info will be obtained
         r = link.rsplit("/", 6) #split link 6 times after every '/'
         product_dict["ID"].append(r[5])

         print("Product ID obtained")
      except NoSuchElementException:
         product_dict["ID"].append("N/A")

      time.sleep(3)
      try: #generate V4 UUID for product and image
         product_uuid = uuid.uuid4()
         image_uuid = uuid.uuid4()
         product_dict["UUID"].append(product_uuid), image_dict["Product UUID"].append(product_uuid)
         image_dict["Image UUID"].append(image_uuid)

         print("UUID generated")
      except NoSuchElementException:
         product_dict["UUID"].append("N/A")
         image_dict["Product UUID"].append("N/A")
         image_dict["Image UUID"].append("N/A")
      
      time.sleep(3)
      try: #download and save product image using product ID as file name
         image_link = f"{r[5]}.png"
         with open(image_link, "wb") as file: #wb = write and binary mode
            img = scraper.driver.find_element(By.XPATH, 
               '//*[@id="main-content"]/article/div[1]/div/div[1]/div[3]/a/figure/img')
            file.write(img.screenshot_as_png)
            time.sleep(3)
         print("Product image downloaded")
         image_dict["Link"].append(image_link)
         print("Image link added")
      except:
         image_dict["Link"].append("N/A")
         print("Product image not downloaded")
      time.sleep(3)
retrieve_data()

Product name obtained
Price obtained
Product description obtained
Product ID obtained
UUID generated
Product image downloaded
Image link added
Product name obtained
Price obtained
Product description obtained
Product ID obtained
UUID generated
Product image downloaded
Image link added
Product name obtained
Price obtained
Product description obtained
Product ID obtained
UUID generated
Product image downloaded
Image link added
Product name obtained
Price obtained
Product description obtained
Product ID obtained
UUID generated
Product image downloaded
Image link added
Product name obtained
Price obtained
Product description obtained
Product ID obtained
UUID generated
Product image downloaded
Image link added


In [10]:
import pandas as pd

pd.DataFrame(product_dict) #displays product dictionary in panda dataframe
#pd.DataFrame(image_dict) #displays image dictionary in panda dataframe

,Link,Product Name,Price,Description,ID,UUID
0,https://store.eu.square-enix-games.com/en_GB/p...,FINAL FANTASY MINI PLUSH: FINAL FANTASY IX ZIDANE,£15.99,Zidane of FINAL FANTASY IX is now available as...,462642,1ace2f12-04c5-4a01-8232-ee2befc67057
1,https://store.eu.square-enix-games.com/en_GB/p...,FINAL FANTASY JIGSAW PUZZLE - FINAL FANTASY VI...,£14.99,Square Enix is proud to offer this 500-piece j...,604828,1b2e0d26-79ff-40d5-963d-29da243aee11
2,https://store.eu.square-enix-games.com/en_GB/p...,FINAL FANTASY VII,from £9.99 £4.99,Steam key issued upon purchase. Activate via y...,603952,b8b4790f-0d7f-4316-804a-4f326ea95be9
3,https://store.eu.square-enix-games.com/en_GB/p...,FINAL FANTASY MAT - CHOCOBO,£23.99,"Amazingly cute mats of Chocobo, Cactuar, and M...",598069,dd268b7b-0078-49b7-bc35-431cca88766a
4,https://store.eu.square-enix-games.com/en_GB/p...,FINAL FANTASY PLUSH [CACTUAR],£22.99,A plush toy of Cactuar and one of the traditio...,306733,15e0f2f9-cbbb-463e-aa2a-ecdaa060d4e2


In [11]:
import os, os.path
import json
from json import JSONEncoder
from uuid import UUID

new_folder = "C:/Users/Zoya/Desktop/AiCoreScripts/Projects/webscraping/raw_data/"
file = "data"
create_file = os.path.join(new_folder, file+".json") #add file type here

#Dealing with no UUID serialization support in json
JSONEncoder_olddefault = JSONEncoder.default
def JSONEncoder_newdefault(self, o):
    if isinstance(o, UUID): return str(o)
    return JSONEncoder_olddefault(self, o)
JSONEncoder.default = JSONEncoder_newdefault


try: #create raw_data folder in current directory - check if folder already exists
    if not os.path.exists(new_folder):
        os.mkdir(new_folder) #create folder if it doesn't exist
        with open(create_file, "w") as fp: #specify path here 
                json.dump(product_dict, fp,  indent=4)
                
    elif os.path.exists(new_folder): #if folder already exists
        with open(create_file, "w") as fp: 
                json.dump(product_dict, fp,  indent=4)
except FileExistsError:
    print("Already exists")

In [12]:
#move files into a new directory/folder
import os, shutil, os.path #shutil - to save it locally

new_folder1 = "images" #create images folder in current directory
sourcepath = "../webscraping"
source = os.listdir(sourcepath)
destinationpath = "../webscraping/images"

try:
 if not os.path.exists(new_folder1):
  os.mkdir(new_folder1)
  for files in source:
      if files.endswith('.png'):
          shutil.move(os.path.join(sourcepath, files), os.path.join(destinationpath, files))    
 elif os.path.exists(new_folder1):
      for files in source:
        if files.endswith('.png'):
            shutil.move(os.path.join(sourcepath, files), os.path.join(destinationpath, files)) 
except FileExistsError:
 print("Already exists")

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import os, shutil, os.path
for link in link_list[3:8]:

    html_page = requests.get(link).content #send get request for each link in range and get content of each request
    soup = BeautifulSoup(html_page)
    images = []
    for img in soup.find_all("picture"): #finds .jpg images only
        
        images.append(img.get("src"))

    print(images)
    #print(img)
    #check = r".jpg"
    #for _ in os.listdir(images):
     #if _.endswith(check):
      #  print(_)
#for img in images:
 #   if img.contains('.jpg'): 
  #      print(img)

[None, None, None]
[None]
[None, None, None, None]
[None, None]
[None]


In [ ]:
import urllib.request

urllib.request.urlretrieve(images, "test.jpg")

TypeError: expected string or bytes-like object